# Modification of DWR Python Notebook for Dry Domestic Well Susceptibility Analysis v1.1
Gabrielle LaMarr LeMee

This is a modification of a Department of Water Resources (DWR) analysis that identifies areas within groundwater basins throughout the State that may be prone to water supply shortages due to domestic drinking water wells going dry. The dashboard identifies the density of "susceptible” domestic wells per square mile based on recent groundwater level measurements and modeled future depth to water. If the modeled future depth to water falls below the dry well depth of a domestic well, the well is labelled susceptible.

DWR notebook: https://data.cnra.ca.gov/dataset/calgw-live/resource/c1c30d5f-c4bf-43fa-94d3-e7c9d5037df9

*Their data note: The dashboard and this underlying analysis provide a density map of domestic wells that are susceptible to going dry if recent groundwater trends continue. The map can be used to evaluate the relative density distribution within groundwater basins. However, the map should not be used to estimate the absolute number of domestic wells that are susceptible to going dry for any area or groundwater level scenario. While the applied groundwater level scenario is based on best available datasets, the scenario is hypothetical, and is chosen to resolve regional differences in the density of domestic wells that are susceptible to going dry. Available groundwater level data are interpolated and projected to domestic wells locations. To achieve near statewide coverage for this analysis, groundwater level measurements are projected up to 10 miles in areas where no local monitoring exists. Filling these groundwater level data gaps with new monitoring stations will improve the reliability of the Dry Well Susceptibility Analysis. The applied model for the depth at which a domestic well is susceptible to going dry is simplified and uncalibrated and is not intended to represent any specific well failure mode. Future calibration of this model, and consideration of well construction details will improve the reliability of the Dry Well Susceptibility Analysis.*

In [114]:
import pandas as pd
import geopandas as gpd
import matplotlib
from datetime import datetime as dt
import numpy as np
import itertools
from itertools import combinations

In [115]:
pd.set_option('display.float_format', '{:.2f}'.format)

## Download Groundwater Level Data

In [116]:
# Read groundwater level
# For dataset info, see https://data.cnra.ca.gov/dataset/periodic-groundwater-level-measurements
url="https://data.cnra.ca.gov/dataset/dd9b15f5-6d08-4d8c-bace-37dc761a9c08/resource/bfa9f262-24a1-45bd-8dc8-138bc8107266/download/measurements.csv"
# If you have already downloaded the measurements dataset, read in the local copy
# url=".../measurements.csv"
df_gwld=pd.read_csv(url)

In [117]:
# Keep only relevant fields
df_gwld = df_gwld[['site_code','msmt_date','gse_gwe','wlm_qa_desc']]

In [118]:
# Convert column types
df_gwld['msmt_date'] = df_gwld['msmt_date'].astype('datetime64[ns]')
df_gwld['gse_gwe'] = df_gwld['gse_gwe'].astype(np.float64)

In [119]:
# Add  measurement Year Column
df_gwld['msmt_year'] = df_gwld['msmt_date'].dt.year

In [120]:
# Only keep data from a specified year onwards
lowcutoffyear = 2011
highcutoffyear = 2021
df_gwld = df_gwld[(df_gwld.msmt_year >= lowcutoffyear) & (df_gwld.msmt_year <= highcutoffyear)]

In [121]:
# Use only those measurements with
# WLM_QA_DESC is NULL (None), or
# WLM_QA_DESC=’Oil or foreign substance in casing’ or
# WLM_QA_DESC=’Acoustical sounder’
df_gwld['wlm_qa_desc'].fillna("No Water Level Measurement Quality Description", inplace = True)
df_gwld=df_gwld[df_gwld['wlm_qa_desc'].isin(['Oil or foreign substance in casing', 'Acoustical sounder', 'No Water Level Measurement Quality Description'])]
df_gwld.drop(columns=['wlm_qa_desc'],inplace=True)

In [122]:
df_gwld.head()

,site_code,msmt_date,gse_gwe,msmt_year
0,320000N1140000W001,2021-08-26 20:00:00,412.00,2021
1,320000N1140000W001,2021-07-29 19:00:00,410.00,2021
2,320000N1140000W001,2021-06-24 18:00:00,411.00,2021
3,320000N1140000W001,2021-05-27 17:00:00,411.00,2021
4,320000N1140000W001,2021-04-29 16:00:00,409.00,2021


## Find Spring Groundwater Levels

In [123]:
# Find spring (high) water levels/minimum depth to water
# Specify Year from which to use Spring WLs
SpringWLYear = 2021
# Only keep measurements from SpringWLYear
df_springWL = df_gwld[df_gwld.msmt_year == SpringWLYear]

In [124]:
# Only keep spring (january-may) measurements
mask_Spring = (df_gwld['msmt_date'].dt.month>=1) & (df_gwld['msmt_date'].dt.month<=5)
df_springWL.loc[mask_Spring, 'msmt_season'] = 'Spring'
df_springWL.dropna(subset=['msmt_season'],inplace=True)

/var/folders/1x/hkrq_4px1_x314p6_8yb37m80000gn/T/ipykernel_2090/409520231.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_springWL.loc[mask_Spring, 'msmt_season'] = 'Spring'
/var/folders/1x/hkrq_4px1_x314p6_8yb37m80000gn/T/ipykernel_2090/409520231.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_springWL.dropna(subset=['msmt_season'],inplace=True)


In [125]:
# Find min. depth to water value for each site code
df_springWL=df_springWL.groupby(['site_code'])['gse_gwe'].min().reset_index().round(2)

In [126]:
# To filter out noise, only use those wells where
# SpringMinDepth IS NOT NULL AND
# SpringMinDepth <1000 AND SpringMinDepth >0
df_springWL = df_springWL[(df_springWL['gse_gwe'] > 0) & (df_springWL['gse_gwe'] < 1000)]

In [127]:
df_springWL.head()

,site_code,gse_gwe
0,320000N1140000W001,409.00
1,325536N1170608W001,13.83
2,325910N1170835W006,12.95
3,326821N1171124W006,8.98
4,327781N1171209W006,18.02


## Find 5-year water level variability

In [129]:
# Find the min/max depth to water value for each year
# To filter out noise, instead of true min/max, use
# 5th and 95th percentile instead
df_gwl_annual_min_max=df_gwld.groupby(['site_code','msmt_year'])['gse_gwe'].describe(percentiles=[.05,.95])[['5%','95%']].reset_index().round(2)
df_gwl_annual_min_max.rename(columns={'5%':'max','95%':'min'}, inplace=True)

In [130]:
# Create unique list of Site COdes
lst_SITE_CODES = df_gwl_annual_min_max['site_code'].unique().tolist()

In [131]:
# Create list of years to consider
lst_Years = [i for i in range(lowcutoffyear,highcutoffyear+1)]

In [132]:
# Create 2D list of all site code and year combinations
lst_all_Site_Codes_and_Years = list(itertools.product(lst_SITE_CODES, lst_Years))

In [133]:
# Convert list of site codes and years to dataframe
df_all_Site_Codes_and_Years = pd.DataFrame (lst_all_Site_Codes_and_Years, columns = ['site_code','msmt_year'])

In [134]:
# Merge dataframe of all site codes and years with actual data table, to
# have rows for every year, even if there is no data, so 
# that rolling min/max can be calculated
df_gwl_annual_min_max_MERGED = pd.merge(df_gwl_annual_min_max, df_all_Site_Codes_and_Years,  how='right', left_on=['msmt_year','site_code'], right_on = ['msmt_year','site_code'])

In [135]:
# To calculate 5 year variability, run rolling min/max, calculate drop, and pick max drop
# Define the time window in years to calculate WL drop from
VariabilityWindowInYears=5
# Limit the maximim drop to be considered (to filter out noisy data)
MaxDrop=300

indexer = pd.api.indexers.FixedForwardWindowIndexer(window_size=VariabilityWindowInYears)
df_gwl_annual_min_max_MERGED['movingMin'] = df_gwl_annual_min_max_MERGED.groupby('site_code')['min'].transform(lambda x: x.rolling(window=indexer, min_periods=VariabilityWindowInYears).max())
df_gwl_annual_min_max_MERGED['movingMax'] = df_gwl_annual_min_max_MERGED.groupby('site_code')['max'].transform(lambda x: x.rolling(window=indexer, min_periods=VariabilityWindowInYears).min())
# calculate rolling 5 year drop
df_gwl_annual_min_max_MERGED['Drop']=df_gwl_annual_min_max_MERGED['movingMin']-df_gwl_annual_min_max_MERGED['movingMax']
# find max 5-year drop for each site
df_Drop=df_gwl_annual_min_max_MERGED.groupby(['site_code'])['Drop'].max().reset_index()
# remove rows with no Drop values
df_Drop.dropna(subset=['Drop'],inplace=True)
# remove rows with noisy/erroneous Drop values
df_Drop = df_Drop[(df_Drop['Drop'] > 0) & (df_Drop['Drop'] < MaxDrop)]

In [136]:
df_Drop.head()

,site_code,Drop
0,320000N1140000W001,57.70
1,325536N1170608W001,17.10
2,325910N1170835W006,5.43
3,326821N1171124W006,6.28
5,327781N1171209W006,2.24


## Download GWL Station data  
and join Spring WL and 5-year level drop attributes

In [138]:
# Read groundwater level stations
# For dataset info, see https://data.cnra.ca.gov/dataset/gspmd/resource/38dc5a77-0428-4d8b-970a-51797ed2cd36
url="https://data.cnra.ca.gov/dataset/536dc423-01b3-4094-bdcd-903df84f6768/resource/38dc5a77-0428-4d8b-970a-51797ed2cd36/download/groundwater_level_sites.csv"
# If you have already downloaded the measurements dataset, read in the local copy
# url=".../stations.csv"
df_gwls=pd.read_csv(url)

In [139]:
df_gwls.head(1)

,STN_ID,SITE_CODE,WELL_NAME,BASIN_NAME,GSA_NAME,GSP_NAME,MONITORING_NETWORK_TYPE,SUSTAINABILITY_INDICATORS,PRINCIPAL_AQUIFER,SWN,...,SMC_START_DATE,SMC_MT,SMC_IM_5_YR,SMC_IM_10_YR,SMC_IM_15_YR,SMC_MO,COMMENTS,FIRST_MSMT_DATE,LAST_MSMT_DATE,MSMT_COUNT
0,2294,344089N1187801W001,04N18W20R01S,4-004.06 Piru,Fillmore and Piru Basins GSA - Piru,NaN,SGMA Representative,"Groundwater Storage, Groundwater Levels",B,04N18W20R001S,...,2022-02-01 00:00:00,344.00,NaN,NaN,NaN,620.00,Basin is not in long term overdraft so no inte...,1972-10-09 00:00:00,2022-05-17 12:00:00,476


In [140]:
# Remove wells with total well depth<100 and top_perf>400
df_gwls = df_gwls[(df_gwls["WELL_DEPTH"] >= 100) | pd.isna(df_gwls["WELL_DEPTH"])]
df_gwls = df_gwls[(df_gwls["TOP_PRF"] >= 400) | pd.isna(df_gwls["TOP_PRF"])]

In [141]:
# Add Spring Depth to Water and 5yr-level WL drop fields
df_gwls = df_gwls[["SITE_CODE","BASIN_NAME","LATITUDE","LONGITUDE"]]
df_gwls.columns = df_gwls.columns.str.lower()
df_merge = df_gwls.merge(df_springWL, on="site_code", how="left")
df_merge = df_merge.merge(df_Drop, on="site_code", how="left")
df_merge = df_merge.dropna(subset=['gse_gwe'])
df_merge.head()

,site_code,basin_name,latitude,longitude,gse_gwe,Drop
0,343939N1187920W001,4-004.06 Piru,34.39,-118.79,66.30,54.52
1,342533N1192690W001,4-004.03 Mound,34.25,-119.27,11.49,19.75
2,343089N1189415W001,4-008 Las Posas Valley,34.31,-118.94,576.30,47.74
3,341242N1191331W001,4-004.02 Oxnard,34.12,-119.13,56.05,90.04
4,341242N1191331W003,4-004.02 Oxnard,34.12,-119.13,51.79,90.94


## Create water level and drops geodataframe

In [142]:
gdf = gpd.GeoDataFrame(df_merge, geometry=gpd.points_from_xy(df_merge.longitude, df_merge.latitude))

In [143]:
gdf.head()

,site_code,basin_name,latitude,longitude,gse_gwe,Drop,geometry
0,343939N1187920W001,4-004.06 Piru,34.39,-118.79,66.30,54.52,POINT (-118.78900 34.39480)
1,342533N1192690W001,4-004.03 Mound,34.25,-119.27,11.49,19.75,POINT (-119.26800 34.25370)
2,343089N1189415W001,4-008 Las Posas Valley,34.31,-118.94,576.30,47.74,POINT (-118.94100 34.30870)
3,341242N1191331W001,4-004.02 Oxnard,34.12,-119.13,56.05,90.04,POINT (-119.13300 34.12430)
4,341242N1191331W003,4-004.02 Oxnard,34.12,-119.13,51.79,90.94,POINT (-119.13300 34.12430)


In [144]:
gdf = gdf.set_crs(epsg=6933)

## Download Well Completion Reports

In [145]:
# Read well completion reports
# For dataset info, see https://data.cnra.ca.gov/dataset/periodic-groundwater-level-measurements
url="https://data.cnra.ca.gov/dataset/647afc02-8954-426d-aabd-eff418d2652c/resource/8da7b93b-4e69-495d-9caa-335691a1896b/download/wellcompletionreports.csv"
# If you have already downloaded the measurements dataset, read in the local copy
# url=".../measurements.csv"
df_wcr=pd.read_csv(url)

/var/folders/1x/hkrq_4px1_x314p6_8yb37m80000gn/T/ipykernel_2090/2676508439.py:6: DtypeWarning: Columns (14,15) have mixed types. Specify dtype option on import or set low_memory=False.
  df_wcr=pd.read_csv(url)


In [146]:
df_wcr.head()

,WCRNUMBER,LEGACYLOGNUMBER,REGIONOFFICE,COUNTYNAME,LOCALPERMITAGENCY,PERMITDATE,PERMITNUMBER,OWNERASSIGNEDWELLNUMBER,WELLLOCATION,CITY,...,CASINGDIAMETER,DRILLINGMETHOD,FLUID,STATICWATERLEVEL,TOTALDRAWDOWN,TESTTYPE,PUMPTESTLENGTH,WELLYIELD,WELLYIELDUNITOFMEASURE,OTHEROBSERVATIONS
0,WCR0237163,E0146494,DWR North Central Region Office,Alameda,Zone 7 Water Agency - Alameda County Flood Con...,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,WCR0245038,01-1476,DWR North Central Region Office,Alameda,"Alameda County Public Works Agency, Water Reso...",NaN,NaN,NaN,9318 CASTRO VALLEY BLVD,HAYWARD,...,NaN,Other,Other,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,WCR0244931,E000126,DWR North Central Region Office,Alameda,Zone 7 Water Agency - Alameda County Flood Con...,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,WCR0245618,14769,DWR North Central Region Office,Alameda,Zone 7 Water Agency - Alameda County Flood Con...,NaN,NaN,NaN,NaN,NaN,...,8.00,Cable Tool,None,NaN,NaN,NaN,NaN,20.00,GPM,NaN
4,WCR0245642,NaN,DWR North Central Region Office,Alameda,"Alameda County Public Works Agency, Water Reso...",NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [147]:
df_wcr = df_wcr[(df_wcr["DATEWORKENDED"] >= "1977-01-01") & (df_wcr["PLANNEDUSEFORMERUSE"].str.contains("Domestic")) & (df_wcr["RECORDTYPE"].isin(['WellCompletion/New/Production or Monitoring/NA', 'WellCompletion/Modification or Repair/Production or Monitoring/NA']))]

## Define the AtRiskWellDepth value

In [148]:
df_wcr["SusceptibleWellDepth"] = df_wcr["TOTALCOMPLETEDDEPTH"] * 0.8

## Find the closest monitoring well for each domestic well

In [149]:
df_wcr["DECIMALLONGITUDE"] = pd.to_numeric(df_wcr["DECIMALLONGITUDE"], errors='coerce')
df_wcr["DECIMALLATITUDE"] = pd.to_numeric(df_wcr["DECIMALLATITUDE"], errors='coerce')

In [150]:
df_wcr = df_wcr.dropna(subset=['DECIMALLONGITUDE', 'DECIMALLATITUDE', 'SusceptibleWellDepth'])

In [151]:
gdf_wcr = gpd.GeoDataFrame(df_wcr, geometry=gpd.points_from_xy(df_wcr.DECIMALLONGITUDE, df_wcr.DECIMALLATITUDE))

In [152]:
gdf_wcr = gdf_wcr.set_crs(epsg=6933)

In [153]:
# Find the closest monitoring well
gdf_wcr_data = gpd.sjoin_nearest(gdf_wcr, gdf, how='left', distance_col="distance_meters")

In [154]:
gdf.columns

Index(['site_code', 'basin_name', 'latitude', 'longitude', 'gse_gwe', 'Drop',
       'geometry'],
      dtype='object')

In [155]:
gdf_wcr_data_trim = gdf_wcr_data[['WCRNUMBER', 'DECIMALLONGITUDE', 'DECIMALLATITUDE', 'SusceptibleWellDepth', 'site_code', 'basin_name', 
                                  'latitude', 'longitude', 'gse_gwe', 'Drop', 'geometry', 'distance_meters']]

## Set flag to identify those wells that are already dry given these assumptions

In [156]:
gdf_wcr_data_trim["IsAlreadyDry"] = False
gdf_wcr_data_trim.loc[gdf_wcr_data_trim['gse_gwe'] >= gdf_wcr_data_trim['SusceptibleWellDepth'], 'IsAlreadyDry'] = True

## Set flag to identify those wells that are susceptible to go dry given these assumptions

In [157]:
gdf_wcr_data_trim["Susceptible"] = False
gdf_wcr_data_trim.loc[(gdf_wcr_data_trim['gse_gwe'] + gdf_wcr_data_trim['Drop']) >= gdf_wcr_data_trim['SusceptibleWellDepth'], 'Susceptible'] = True

In [158]:
gdf_wcr_data_trim.head()

,WCRNUMBER,DECIMALLONGITUDE,DECIMALLATITUDE,SusceptibleWellDepth,site_code,basin_name,latitude,longitude,gse_gwe,Drop,geometry,distance_meters,IsAlreadyDry,Susceptible
367464,WCR2015-000207,-1233.55,39.80,108.00,392358N1232020W001,1-052 Ukiah Valley,39.24,-123.20,17.40,34.25,POINT (-1233.551 39.796),1110.35,False,False
588625,WCR1977-010789,-114.24,34.19,44.00,339034N1167436W001,7-021.04 San Gorgonio Pass,33.90,-116.74,311.88,20.78,POINT (-114.238 34.190),2.52,True,True
199072,WCR1971-002900,-114.48,32.88,68.80,330924N1169465W001,9-010 San Pasqual Valley,33.09,-116.95,64.65,45.34,POINT (-114.478 32.884),2.48,False,True
609450,WCR1969-002005,-114.31,34.16,160.00,339034N1167436W001,7-021.04 San Gorgonio Pass,33.90,-116.74,311.88,20.78,POINT (-114.308 34.160),2.45,True,True
597169,WCR1969-002007,-114.31,34.16,80.00,339034N1167436W001,7-021.04 San Gorgonio Pass,33.90,-116.74,311.88,20.78,POINT (-114.308 34.160),2.45,True,True


## What does this look like for not yet dry wells?
On average, domestic wells that are not yet dry can tolerate 142 feet of water level decline. If conditions for the next 5 years are similar to the previous 5 years, these wells face an average decline of 27 feet.

In [159]:
not_dry = gdf_wcr_data_trim.loc[gdf_wcr_data_trim["IsAlreadyDry"] == False]

In [160]:
len(not_dry)

234621

In [161]:
not_dry["difference"] = not_dry["SusceptibleWellDepth"] - not_dry["gse_gwe"]

/Users/glamarrlemee/.local/share/virtualenvs/dwr-well-susceptibility-M0lscrqj/lib/python3.9/site-packages/geopandas/geodataframe.py:1443: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [162]:
not_dry[['difference']].describe()

,difference
count,234621.00
mean,142.43
std,129.41
min,0.00
25%,54.44
50%,106.60
75%,188.58
max,1592.00


In [163]:
not_dry[['Drop']].describe()

,Drop
count,196624.00
mean,27.18
std,24.04
min,1.12
25%,11.37
50%,22.20
75%,32.31
max,281.44


## What does this look like for susceptible wells?
10% of not-yet-dry wells are susceptible to going dry in the next 5 years if water level declines are similar to the previous 5 years. These susceptible wells have, on average, 24 feet of depth remaining and face a 49 foot decline. 

In [164]:
susceptible = not_dry.loc[not_dry["Susceptible"] == True]

In [165]:
len(susceptible)

24119

In [166]:
len(susceptible) / len(not_dry)

0.10279983462690893

In [167]:
susceptible[['difference']].describe()

,difference
count,24119.00
mean,24.17
std,24.62
min,0.00
25%,8.12
50%,16.75
75%,32.90
max,278.21


In [168]:
susceptible[['Drop']].describe()

,Drop
count,24119.00
mean,49.23
std,37.60
min,2.72
25%,23.52
50%,37.08
75%,72.40
max,281.44
